# Import Library 

In [2]:
import  cassandra
import pandas as pd
import glob, csv, re, json, os
import numpy as np 

# Creating List Path 

In [2]:
# Check current dir 
print(f"current working dir {os.getcwd()}")

current working dir /home/hira/Data-Modeling-with-Cassandra


In [7]:
file_path = os.getcwd() + '/event_data'
file_path

'/home/hira/Data-Modeling-with-Cassandra/event_data'

In [10]:
for root, dir, files in  os.walk(file_path):
    filepath_list = glob.glob(os.path.join(root,"*-events.csv"))

filepath_list

['/home/hira/Data-Modeling-with-Cassandra/event_data/2018-11-16-events.csv',
 '/home/hira/Data-Modeling-with-Cassandra/event_data/2018-11-22-events.csv',
 '/home/hira/Data-Modeling-with-Cassandra/event_data/2018-11-30-events.csv',
 '/home/hira/Data-Modeling-with-Cassandra/event_data/2018-11-27-events.csv',
 '/home/hira/Data-Modeling-with-Cassandra/event_data/2018-11-06-events.csv',
 '/home/hira/Data-Modeling-with-Cassandra/event_data/2018-11-12-events.csv',
 '/home/hira/Data-Modeling-with-Cassandra/event_data/2018-11-28-events.csv',
 '/home/hira/Data-Modeling-with-Cassandra/event_data/2018-11-13-events.csv',
 '/home/hira/Data-Modeling-with-Cassandra/event_data/2018-11-10-events.csv',
 '/home/hira/Data-Modeling-with-Cassandra/event_data/2018-11-11-events.csv',
 '/home/hira/Data-Modeling-with-Cassandra/event_data/2018-11-24-events.csv',
 '/home/hira/Data-Modeling-with-Cassandra/event_data/2018-11-21-events.csv',
 '/home/hira/Data-Modeling-with-Cassandra/event_data/2018-11-05-events.csv',

In [9]:
# initiating empty list  rows that will be generated  data 

full_data_rows = []

# for every  filepath in the file of list
for file in filepath_list:
    # reading csv 
    with open(file, 'r', encoding= 'utf8', newline ='')  as  csvfile:
         csvreader = csv.reader(csvfile)
         next(csvreader)
         
         # extracting each data row one by one and append it
         for line in csvreader:
             full_data_rows.append(line)
print(f"Total rows: {len(full_data_rows)}")
print(f"Sample data : \n {full_data_rows[:5]}")


Total rows: 8056
Sample data : 
 [['Mudhoney', 'Logged In', 'Aleena', 'F', '10', 'Kirby', '231.57506', 'paid', 'Waterloo-Cedar Falls, IA', 'PUT', 'NextSong', '1.54102E+12', '637', 'Get Into Yours', '200', '1.54233E+12', '44'], ['Carpenters', 'Logged In', 'Aleena', 'F', '11', 'Kirby', '238.39302', 'paid', 'Waterloo-Cedar Falls, IA', 'PUT', 'NextSong', '1.54102E+12', '637', 'Yesterday Once More', '200', '1.54233E+12', '44'], ['', 'Logged In', 'Aleena', 'F', '12', 'Kirby', '', 'paid', 'Waterloo-Cedar Falls, IA', 'GET', 'Home', '1.54102E+12', '637', '', '200', '1.54233E+12', '44'], ['Kings Of Leon', 'Logged In', 'Aleena', 'F', '13', 'Kirby', '185.28608', 'paid', 'Waterloo-Cedar Falls, IA', 'PUT', 'NextSong', '1.54102E+12', '637', 'Taper Jean Girl', '200', '1.54233E+12', '44'], ['The Bloody Beetroots', 'Logged In', 'Aleena', 'F', '14', 'Kirby', '201.97832', 'paid', 'Waterloo-Cedar Falls, IA', 'PUT', 'NextSong', '1.54102E+12', '637', 'Warp 1.9 (feat. Steve Aoki)', '200', '1.54233E+12', '44']

In [6]:
    """
    creating a smaller event data csv file  called event_datafile_full csv that will be
    inser into apache cassandra   
    """
    
csv.register_dialect('myDialect', quoting = csv.QUOTE_ALL,\
                     skipinitialspace = True)

In [7]:
column = ['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId']

In [9]:
 with open('event_datefile_new.csv', 'w', encoding='utf8'\
     , newline='') as f :
        writer = csv.writer(f, dialect= 'myDialect')
        writer.writerow(column)
        for row in  full_data_rows:
            if row[0] == '':
                continue
            writer.writerow((row[0], row[2], row[3],row[4],row[5],row[6],row[7],row[8],row[12],row[13],row[16]))

In [12]:
with open('event_datefile_new.csv', 'r', encoding='utf8')  as f :
    print(sum( 1 for line in f ))

6821


# Event *datefile_new.csv*  are ready to use 
Meta data :

| Table Name      | Description |
| ----------- | ----------- |
|artist  	 | 	  Name artist |
|firstName  	 | first name user 	 |
|gender  	 | 	user gender |
|itemInSession  	 |  session user use apps	 |
|lastName  	 | 	last name user |
|length  	 | length user stream music	 |
|level  	 |  paid or free	 |
|location  	 |  location user	 |
|sessionId  	 | sesion id user 	 |
|song  	 | 	 song play |
|userId  	 |  user id	 |

# Creating Cluster 

In [3]:
"""
This should connected to  a cassandra  instance your local machine
"""
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['172.17.0.2'])
    session = cluster.connect()
    print("Connected Estabilished !!")
except Exception  as e :
    print(f"Connected filed ! error : {e}")

Connected Estabilished !!


# Creating Keyspace 

In [34]:
keyspace_quer ="""
                    CREATE KEYSPACE IF NOT EXISTS sparkify 
                    WITH REPLICATION = 
                    {'class' :'SimpleStrategy' ,'replication_factor': 1}
                """

try: 
    session.execute(keyspace_quer)
except Exception as e:
    print(f"Failed create keyspace : {e}")


In [35]:

# creating keyspace
session.set_keyspace('sparkify')

   # Create Table session_item 
* We need a way to run  query on sessionId and iteminSession. So our primary key  must have these columns. We can partitiom the data on sessionId

* Our Select query : SELECT artist, song, length FROM  session_item where sessionId = 338 and itemInSession = 4
*  Our Primary key will be (sessionId, itemInSession), where sessionId is the partition key and  itemInSession is the clustering column.



In [37]:
# Create table session_item   
create_query1 = """
                CREATE TABLE IF NOT EXISTS session_item (
                artist text,
                song text,
                length float,
                sessionid int,
                iteminsession int,
                PRIMARY KEY(sessionid,iteminsession)
                )
                """

try:
    session.execute(create_query1)
    print("Table created")
except Exception as e :
    print(f"Failde Create table {e}")

Table created


In [42]:
# using the event file 
file = "event_datefile_new.csv"

# reading csv andn inserting rows  into cassandra table
with open(file, encoding='utf8') as f :
    csvreader = csv.reader(f)
    next(csvreader) # skup header
    # insert into tables
    for line in csvreader:
        query = "INSERT INTO session_item (artist, song, length, sessionid, iteminsession)"
        query = query + "VALUES (%s,%s,%s,%s,%s)"
        session.execute(query, (line[0], line[10],float( line[5]),int(line[8]) , int(line[3])))

# Select Statement

In [44]:
Select_statement = " SELECT * FROM  session_item  limit 1"

try :
    rows = session.execute(Select_statement)
except Exception as e :
    print(e)
    
for row in rows:
    print(row)

Row(sessionid=23, iteminsession=0, artist='Regina Spektor', length=191.08526611328125, song='24')


# Create Table user_session
* Our Primary key will be ((sessionId, userId), itemInSession)), where (sessionId, userId) is the partition key and itemInSession is the clustering column.
* Also, we are using the clause - WITH CLUSTERING ORDER BY (itemInSession ASC), to sort our data based on itemInSession
* Columns we included in the table : sessionId, userId, artist, song, firstName, lastName, itemInSession

In [47]:
b

Table created


In [48]:
print(session.execute("DESCRIBE TABLE user_session"))

In [59]:
# using the event file 
file = "event_datefile_new.csv"

# reading csv andn inserting rows  into cassandra table
with open(file, encoding='utf8') as f :
    csvreader = csv.reader(f)
    next(csvreader) # skup header
    # insert into tables
    for line in csvreader:
        query = "INSERT INTO user_session ( sessionid,userid,artist,song,firstname,lastname,iteminsession )"
        query = query + "VALUES (%s,%s,%s,%s,%s, %s, %s)"
        session.execute(query, (int(line[-3]),int( line[-1]), line[0], line[-2], line[1] , line[4], int(line[3])))

# Select Statement user_session tables 

In [60]:
Select_statement = " SELECT * FROM  user_session  limit 1"

try :
    rows = session.execute(Select_statement)
except Exception as e :
    print(e)
    
for row in rows:
    print(row)

Row(sessionid=1049, userid=73, iteminsession=0, artist='Manowar', firstname='Jacob', lastname='Klein', song='Shell Shock')


# Create Table user_song 
* Select query : SELECT song, firstName, lastName FROM user_song where song = 'All Hands Against His Own'
* Our Primary key will be ((song), userId)), where song is the partition key and  userId is the clustering column.
* Columns we included in the table : song, userId, firstName, lastName

In [63]:
# Create table suser_session

create_query1 = """
                CREATE TABLE IF NOT EXISTS user_song (
                song text ,
                userid int,
                firstname text,
                lastname text,
                
                PRIMARY KEY((song),userid))
                
                
                """

try:
    session.execute(create_query1)
    print("Table created")
except Exception as e :
    print(f"Failde Create table {e}")

Table created


In [65]:
# using the event file 
file = "event_datefile_new.csv"

# reading csv andn inserting rows  into cassandra table
with open(file, encoding='utf8') as f :
    csvreader = csv.reader(f)
    next(csvreader) # skup header
    # insert into tables
    for line in csvreader:
        query = "INSERT INTO user_song ( song,userid,firstname,lastname )"
        query = query + "VALUES (%s,%s,%s,%s)"
        session.execute(query, ( line[-2], int(line[-1]) , line[1], line[4]))

In [66]:
Select_statement = " SELECT * FROM  user_song  limit 1"

try :
    rows = session.execute(Select_statement)
except Exception as e :
    print(e)
    
for row in rows:
    print(row)

Row(song="Wonder What's Next", userid=49, firstname='Chloe', lastname='Cuevas')


# Drop The table before closing

In [69]:
tablesinfo = session.execute("SELECT * FROM system_schema.tables WHERE keyspace_name ='sparkify'")
tables_list = [ table[1]  for table in tablesinfo]
tables_list

['session_item', 'user_session', 'user_song']

In [73]:
def delete_tables (tables):
    for table in tables:
        try :
            sql = f"DROP TABLE IF EXISTS sparkify.{table}"
            session.execute(sql)
            print( f"drop table {table} finish "  )
        except Exception as e :
            print(e)

In [74]:
delete_tables(tables_list)


drop table session_item finish 
drop table user_session finish 
drop table user_song finish 


# Close The Session  and cluster connection

In [76]:
session.shutdown()
cluster.shutdown()